# 第1章: グラフの基本要素


## 準備

以下のセルを順番に実行して、演習に必要な環境をセットアップします。


### LLMプロバイダーの選択

このセルでは、使用するLLMプロバイダーを選択します。
`LLM_PROVIDER` 変数に、利用したいプロバイダー名を設定してください。
選択可能なプロバイダー: `"openai"`, `"azure"`, `"google"` (Vertex AI), `"google_genai"` (Gemini API), `"anthropic"`, `"bedrock"`


In [ ]:
# === LLMプロバイダーの選択 ===
# 利用したいLLMプロバイダーを以下の変数で指定してください。
# "openai", "azure", "google" (Vertex AI), "google_genai" (Gemini API), "anthropic", "bedrock" のいずれかを選択できます。
LLM_PROVIDER = "openai"  # 例: OpenAI を利用する場合


### APIキー/環境変数の設定

以下のセルを実行する前に、選択したLLMプロバイダーに応じたAPIキーまたは環境変数を設定する必要があります。

**手順:**
1.  `.env.sample` ファイルをコピーして `.env` ファイルを作成します。
2.  `.env` ファイルを開き、選択したLLMプロバイダーに対応するAPIキーや必要な情報を記述します。
    *   **OpenAI:** `OPENAI_API_KEY`
    *   **Azure OpenAI:** `AZURE_OPENAI_API_KEY`, `AZURE_OPENAI_ENDPOINT`, `OPENAI_API_VERSION`, `AZURE_OPENAI_DEPLOYMENT_NAME`
    *   **Google (Vertex AI):** `GOOGLE_CLOUD_PROJECT_ID`, `GOOGLE_CLOUD_LOCATION` (Colab環境外で実行する場合、`GOOGLE_APPLICATION_CREDENTIALS` 環境変数の設定も必要になることがあります)
    *   **Google (Gemini API):** `GOOGLE_API_KEY`
    *   **Anthropic:** `ANTHROPIC_API_KEY`
    *   **AWS Bedrock:** `AWS_ACCESS_KEY_ID`, `AWS_SECRET_ACCESS_KEY`, `AWS_REGION_NAME` (IAMロールを使用する場合は、これらのキー設定は不要な場合がありますが、リージョン名は必須です)
3.  ファイルを保存します。

**Google Colab を使用している場合:**
上記の `.env` ファイルを使用する代わりに、Colabのシークレットマネージャーに必要なキーを登録してください。
例えば、OpenAIを使用する場合は `OPENAI_API_KEY` という名前でシークレットを登録します。
Vertex AI を利用する場合は、Colab上での認証 (`google.colab.auth.authenticate_user()`) が実行されます。

このセルは、設定された情報に基づいて環境変数をロードし、LLMクライアントを初期化します。


In [ ]:
# === APIキー/環境変数の設定 ===
import os
from dotenv import load_dotenv

# .envファイルから環境変数を読み込む (存在する場合)
load_dotenv()

try:
    from google.colab import userdata
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

# --- OpenAI ---
if LLM_PROVIDER == "openai":
    OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
    if not OPENAI_API_KEY and IS_COLAB:
        OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
    if not OPENAI_API_KEY:
        raise ValueError("OpenAI APIキーが設定されていません。環境変数 OPENAI_API_KEY を設定するか、Colab環境の場合はシークレットに OPENAI_API_KEY を設定してください。")
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# --- Azure OpenAI ---
elif LLM_PROVIDER == "azure":
    AZURE_OPENAI_API_KEY = os.environ.get("AZURE_OPENAI_API_KEY")
    AZURE_OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
    OPENAI_API_VERSION = os.environ.get("OPENAI_API_VERSION")
    AZURE_OPENAI_DEPLOYMENT_NAME = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")

    if IS_COLAB:
        if not AZURE_OPENAI_API_KEY: AZURE_OPENAI_API_KEY = userdata.get("AZURE_OPENAI_API_KEY")
        if not AZURE_OPENAI_ENDPOINT: AZURE_OPENAI_ENDPOINT = userdata.get("AZURE_OPENAI_ENDPOINT")
        if not OPENAI_API_VERSION: OPENAI_API_VERSION = userdata.get("OPENAI_API_VERSION") # 例: "2023-07-01-preview"
        if not AZURE_OPENAI_DEPLOYMENT_NAME: AZURE_OPENAI_DEPLOYMENT_NAME = userdata.get("AZURE_OPENAI_DEPLOYMENT_NAME")

    if not AZURE_OPENAI_API_KEY: raise ValueError("Azure OpenAI APIキー (AZURE_OPENAI_API_KEY) が設定されていません。")
    if not AZURE_OPENAI_ENDPOINT: raise ValueError("Azure OpenAI エンドポイント (AZURE_OPENAI_ENDPOINT) が設定されていません。")
    if not OPENAI_API_VERSION: OPENAI_API_VERSION = "2023-07-01-preview" # デフォルトを設定することも可能
    if not AZURE_OPENAI_DEPLOYMENT_NAME: raise ValueError("Azure OpenAI デプロイメント名 (AZURE_OPENAI_DEPLOYMENT_NAME) が設定されていません。")

    os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
    os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
    os.environ["OPENAI_API_VERSION"] = OPENAI_API_VERSION

# --- Google Cloud Vertex AI (Gemini) ---
elif LLM_PROVIDER == "google":
    PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT_ID") # .env 用に修正
    LOCATION = os.environ.get("GOOGLE_CLOUD_LOCATION")

    if IS_COLAB:
        if not PROJECT_ID: PROJECT_ID = userdata.get("GOOGLE_CLOUD_PROJECT_ID")
        if not LOCATION: LOCATION = userdata.get("GOOGLE_CLOUD_LOCATION") # 例: "us-central1"
        from google.colab import auth as google_auth
        google_auth.authenticate_user() # Vertex AI を使う場合は Colab での認証を推奨
    else: # Colab外の場合、.envから読み込んだ値で環境変数を設定
        if PROJECT_ID: os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID # Vertex AI SDKが参照する標準的な環境変数名
        if LOCATION: os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION

    if not PROJECT_ID: raise ValueError("Google Cloud Project ID が設定されていません。環境変数 GOOGLE_CLOUD_PROJECT_ID を設定するか、Colab環境の場合はシークレットに GOOGLE_CLOUD_PROJECT_ID を設定してください。")
    if not LOCATION: LOCATION = "us-central1" # デフォルトロケーション

# --- Google Gemini API (langchain-google-genai) ---
elif LLM_PROVIDER == "google_genai":
    GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY and IS_COLAB:
        GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY:
        raise ValueError("Google APIキーが設定されていません。環境変数 GOOGLE_API_KEY を設定するか、Colab環境の場合はシークレットに GOOGLE_API_KEY を設定してください。")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# --- Anthropic (Claude) ---
elif LLM_PROVIDER == "anthropic":
    ANTHROPIC_API_KEY = os.environ.get("ANTHROPIC_API_KEY")
    if not ANTHROPIC_API_KEY and IS_COLAB:
        ANTHROPIC_API_KEY = userdata.get("ANTHROPIC_API_KEY")
    if not ANTHROPIC_API_KEY:
        raise ValueError("Anthropic APIキーが設定されていません。環境変数 ANTHROPIC_API_KEY を設定するか、Colab環境の場合はシークレットに ANTHROPIC_API_KEY を設定してください。")
    os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY

# --- Amazon Bedrock (Claude) ---
elif LLM_PROVIDER == "bedrock":
    AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID")
    AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")
    AWS_REGION_NAME = os.environ.get("AWS_REGION_NAME")

    if IS_COLAB: 
        if not AWS_ACCESS_KEY_ID: AWS_ACCESS_KEY_ID = userdata.get("AWS_ACCESS_KEY_ID")
        if not AWS_SECRET_ACCESS_KEY: AWS_SECRET_ACCESS_KEY = userdata.get("AWS_SECRET_ACCESS_KEY")
        if not AWS_REGION_NAME: AWS_REGION_NAME = userdata.get("AWS_REGION_NAME")

    if not AWS_REGION_NAME:
         raise ValueError("AWSリージョン名 (AWS_REGION_NAME) が設定されていません。Bedrock利用にはリージョン指定が必要です。")

    # 環境変数に設定 (boto3がこれらを自動で読み込む)
    if AWS_ACCESS_KEY_ID: os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
    if AWS_SECRET_ACCESS_KEY: os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY
    os.environ["AWS_DEFAULT_REGION"] = AWS_REGION_NAME # boto3が参照する標準的なリージョン環境変数名
    os.environ["AWS_REGION"] = AWS_REGION_NAME # いくつかのライブラリはこちらを参照することもある

print(f"APIキー/環境変数の設定完了 (プロバイダー: {LLM_PROVIDER})")


### LLMクライアントの初期化

このセルは、上で選択・設定したLLMプロバイダーに基づいて、対応するLLMクライアントを初期化します。


In [ ]:
# === LLMクライアントの動的初期化 ===
llm = None

if LLM_PROVIDER == "openai":
    from langchain_openai import ChatOpenAI
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
elif LLM_PROVIDER == "azure":
    from langchain_openai import AzureChatOpenAI
    llm = AzureChatOpenAI(
        azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME"), # 環境変数から取得
        openai_api_version=os.environ.get("OPENAI_API_VERSION"), # 環境変数から取得
        temperature=0,
    )
elif LLM_PROVIDER == "google":
    from langchain_google_vertexai import ChatVertexAI
    # PROJECT_ID, LOCATION は前のセルで環境変数に設定済みか、Colabの場合は直接利用
    llm = ChatVertexAI(model_name="gemini-2.0-flash", temperature=0, project=os.environ.get("GOOGLE_CLOUD_PROJECT"), location=os.environ.get("GOOGLE_CLOUD_LOCATION"))
elif LLM_PROVIDER == "google_genai":
    from langchain_google_genai import ChatGoogleGenerativeAI
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
elif LLM_PROVIDER == "anthropic":
    from langchain_anthropic import ChatAnthropic
    llm = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0)
elif LLM_PROVIDER == "bedrock":
    from langchain_aws import ChatBedrock # langchain_community.chat_models から langchain_aws に変更の可能性あり
    # AWS_REGION_NAME は前のセルで環境変数 AWS_DEFAULT_REGION に設定済み
    llm = ChatBedrock( # BedrockChat ではなく ChatBedrock が一般的
        model_id="anthropic.claude-3-haiku-20240307-v1:0",
        # region_name=os.environ.get("AWS_DEFAULT_REGION"), # 通常、boto3が環境変数から自動で読み込む
        model_kwargs={"temperature": 0},
    )
else:
    raise ValueError(
        f"Unsupported LLM_PROVIDER: {LLM_PROVIDER}. "
        "Please choose from 'openai', 'azure', 'google', 'google_genai', 'anthropic', or 'bedrock'."
    )

print(f"LLM Provider: {LLM_PROVIDER}")
if llm:
    print(f"LLM Client Type: {type(llm)}")
    # モデル名取得の試行を汎用的に
    model_attr = (
                 getattr(llm, 'model', None) or
                 getattr(llm, 'model_name', None) or
                 getattr(llm, 'model_id', None) or
                 (hasattr(llm, 'llm') and getattr(llm.llm, 'model', None)) # 一部のLLMクライアントのネスト構造に対応
    )
    if hasattr(llm, 'azure_deployment') and not model_attr: # Azure特有の属性
        model_attr = llm.azure_deployment
        
    if model_attr:
        print(f"LLM Model: {model_attr}")
    else:
        print("LLM Model: (Could not determine model name from client attributes)")


---


### ■ 問題001: 最小構成のLangGraphグラフの構築

LangGraphの最も基本的な構成要素である`StateGraph`と`State`を理解し、シンプルなグラフを構築してみましょう。この問題では、入力された文字列をそのまま出力するだけの、単一のノードを持つグラフを作成します。

*   **学習内容:** この問題では、`StateGraph`、`TypedDict`を用いた`State`の定義、`add_node`、`set_entry_point`、`add_edge`、`END`といったLangGraphの最も基本的なAPIを学びます。また、`Annotated`と`add_messages`を使ってメッセージ履歴を管理する方法も理解します。


In [ ]:
# 解答欄001

from typing import TypedDict, Annotated
from langgraph.graph import ____, END
from langgraph.graph.message import add_messages

# --- 状態定義 (State) ---
class GraphState(____):
    messages: ____[list, add_messages]

# --- ノード定義 (Nodes) ---
def simple_node(state: GraphState):
    return {"messages": [state["messages"][-1]]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(____)

# ノードの追加
workflow.add_node("simple_node", simple_node)

# エントリポイントの設定
workflow.set_entry_point("simple_node")

# 終了ポイントの設定
workflow.add_edge("simple_node", ____)

# グラフのコンパイル
graph = workflow.compile()

# --- グラフの実行と結果表示 ---
inputs = {"messages": [("user", "Hello, LangGraph!")]}

# 最終結果の確認
final_state = graph.invoke(inputs)
print(f"最終的な応答: {final_state['messages'][-1].content}")


<details><summary>解答001</summary>

``````python
# 解答001

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    # グラフの状態を保持する辞書
    # ここでは、入力メッセージを保持する
    messages: Annotated[list, add_messages]

# --- ノード定義 (Nodes) ---
def simple_node(state: GraphState):
    # 入力されたメッセージをそのまま返すノード
    return {"messages": [state["messages"][-1]]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(GraphState)

# ノードの追加
workflow.add_node("simple_node", simple_node)

# エントリポイントの設定
workflow.set_entry_point("simple_node")

# 終了ポイントの設定
workflow.add_edge("simple_node", END)

# グラフのコンパイル
graph = workflow.compile()

# --- グラフの実行と結果表示 ---
inputs = {"messages": [("user", "Hello, LangGraph!")]}

# 最終結果の確認
final_state = graph.invoke(inputs)
print(f"最終的な応答: {final_state['messages'][-1].content}")
``````
</details>


<details><summary>解説001</summary>

#### この問題のポイント

*   **コード解説:**
    *   `GraphState`は、グラフ全体で共有される状態を定義します。`TypedDict`を使うことで、状態のスキーマを明確にできます。`messages: Annotated[list, add_messages]`は、LangChainのメッセージ形式のリストを状態として持ち、新しいメッセージが追加されるたびに自動的にリストの末尾に追加されるように設定しています。
    *   `simple_node`関数は、グラフのノードとして機能します。`state`引数として現在のグラフの状態を受け取り、新しい状態を辞書として返します。ここでは、入力された最後のメッセージをそのまま返しています。
    *   `StateGraph(GraphState)`でグラフのインスタンスを作成し、`GraphState`で定義した状態スキーマを渡します。
    *   `workflow.add_node("simple_node", simple_node)`で、`simple_node`関数を`simple_node`という名前のノードとしてグラフに追加します。
    *   `workflow.set_entry_point("simple_node")`は、グラフの実行が開始される最初のノードを指定します。
    *   `workflow.add_edge("simple_node", END)`は、`simple_node`の実行が完了したらグラフを終了することを示します。`END`はLangGraphが提供する特別な終了ノードです。
    *   `graph = workflow.compile()`で、定義したワークフローを実行可能なアプリケーションにコンパイルします。
    *   `graph.stream(inputs)`は、グラフの実行過程をストリーミングで受け取ることができます。`graph.invoke(inputs)`は、グラフの実行が完了した最終状態を返します。
---
</details>


### ■ 問題002: グラフの可視化

問題001で構築した最小構成のグラフの構造を、視覚的に確認する方法を学びましょう。

*   **学習内容:** `graph.get_graph().draw_png()` を使用して、コンパイル済みのLangGraphグラフ構造をPNG画像として描画し、Jupyter Notebook上に表示する方法を学びます。これにより、グラフのノードとエッジの接続関係を直感的に理解できるようになります。


In [ ]:
# 解答欄002

# 問題001のコードを再掲（このセルでグラフを定義・コンパイルします）
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage

# --- 状態定義 (State) ---
class GraphState(____):
    messages: ____[list, add_messages]

# --- ノード定義 (Nodes) ---
def simple_node(state: GraphState):
    # このノードは状態を更新せず、最後のメッセージをログに出力するだけ
    # LangGraphでは、ノードがNoneまたは空の辞書を返すと、状態は更新されない
    return

# --- グラフ構築 (Graph) ---
workflow = ____(GraphState)
workflow.add_node("simple_node", simple_node)
workflow.set_entry_point(____)
workflow.add_edge("simple_node", ____)
graph = workflow.compile()


In [ ]:
# --- グラフの可視化 ---
from IPython.display import Image, display

try:
    png_data = graph.get_graph().draw_png()
    display(Image(png_data))
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")


In [ ]:
# --- グラフの動作確認 ---
# 可視化したグラフが問題001と同様に動作することを確認します。
try:
    inputs = {"messages": [HumanMessage(content="Hello, this is a test.")]}
    final_state = graph.invoke(inputs)
    print("\nグラフの実行が完了しました。")
    print(f"最終的なmessagesの状態: {final_state['messages']}")
except NameError:
    print("グラフが定義されていません。前のセルを先に実行してください。")


<details><summary>解答002</summary>

``````python
# 解答002

# 問題001のコードを再掲（このセルでグラフを定義・コンパイルします）
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]

# --- ノード定義 (Nodes) ---
def simple_node(state: GraphState):
    # このノードは状態を更新せず、最後のメッセージをログに出力するだけ
    # LangGraphでは、ノードがNoneまたは空の辞書を返すと、状態は更新されない
    return

# --- グラフ構築 (Graph) ---
workflow = StateGraph(GraphState)
workflow.add_node("simple_node", simple_node)
workflow.set_entry_point("simple_node")
workflow.add_edge("simple_node", END)
graph = workflow.compile()

# --- グラフの可視化 ---
from IPython.display import Image, display

try:
    png_data = graph.get_graph().draw_png()
    display(Image(png_data))
    print("グラフが正常に可視化されました。")
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")

# --- グラフの動作確認 ---
# 可視化したグラフが問題001と同様に動作することを確認します。
try:
    inputs = {"messages": [HumanMessage(content="Hello, this is a test.")]}
    final_state = graph.invoke(inputs)
    print("\nグラフの実行が完了しました。")
    print(f"最終的なmessagesの状態: {final_state['messages']}")
except NameError:
    print("グラフが定義されていません。前のセルを先に実行してください。")
``````
</details>


<details><summary>解説002</summary>

#### この問題のポイント

*   **コード解説:**
    *   この問題では、まず問題001で作成したグラフ定義のコードを再利用して、`graph`オブジェクトを準備します。
    *   グラフを可視化するための中心的なメソッドが `graph.get_graph().draw_png()` です。
        *   `graph.get_graph()`: コンパイル済みの`graph`オブジェクトから、可視化や解析が可能な内部グラフ表現を取得します。
        *   `.draw_png()`: 取得したグラフ表現をPNG形式の画像データ（バイト列）として描画します。
    *   `IPython.display.Image` は、画像データをJupyter NotebookなどのIPython環境で表示可能なオブジェクトに変換します。
    *   `IPython.display.display()` 関数を使って、`Image`オブジェクトをセルに表示します。
    *   **重要:** `.draw_png()` メソッドを使用するには、**Graphviz**というグラフ可視化ソフトウェアがシステムにインストールされている必要があります。また、Pythonライブラリの`pygraphviz`も必要です（これらは準備セクションでインストール済みです）。もし`ExecutableNotFound`のようなエラーが出る場合は、Graphviz本体がOSに正しくインストールされ、パスが通っているかを確認してください。

*   **なぜ可視化が重要か:**
    *   グラフが単純なうちはコードを読むだけで構造を理解できますが、ノードやエッジが増え、特に条件分岐やループが絡んでくると、全体の流れを把握するのが難しくなります。
    *   グラフを可視化することで、設計した通りの構造になっているかを一目で確認でき、意図しない接続やループのデバッグに非常に役立ちます。

---
</details>


### ■ 問題003: 複数のノードを持つシーケンシャルグラフの構築

前の問題で学んだ基本的なグラフ構築に加えて、複数のノードを直列に接続し、データがノード間をどのように流れるかを理解しましょう。ここでは、入力された文字列を加工する2つのノード（例：大文字化、逆順化）を持つグラフを作成します。

*   **学習内容:** 複数のノードを`add_edge`で直列に接続する方法と、ノード間で状態がどのように引き継がれるかを学びます。`HumanMessage`と`AIMessage`を使って、メッセージの送信元を明示する方法も理解します。


In [ ]:
# 解答欄003

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, ____

# --- 状態定義 (State) ---
class GraphState(____):
    messages: Annotated[list, add_messages]

# --- ノード定義 (Nodes) ---
def uppercase_node(state: GraphState):
    last_message_content = state["messages"][-1].content
    return {"messages": [AIMessage(content=last_message_content.upper())]}

def reverse_node(state: GraphState):
    last_message_content = state["messages"][-1].content
    return {"messages": [AIMessage(content=last_message_content[::-1])]}

# --- グラフ構築 (Graph) ---
workflow = ____(GraphState)

# ノードの追加
workflow.add_node("uppercase", uppercase_node)
workflow.add_node("reverse", reverse_node)

# エントリポイントの設定
workflow.set_entry_point(____)

# エッジの追加 (直列接続)
workflow.add_edge(____, "reverse")
workflow.add_edge("reverse", END)

# グラフのコンパイル
graph = workflow.compile()


In [ ]:
# --- グラフの可視化 ---
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")


In [ ]:
# --- グラフの実行と結果表示 ---
inputs = {"messages": [HumanMessage(content="Hello LangGraph")]}

final_state = graph.invoke(inputs)
print(f"最終的な応答: {final_state['messages'][-1].content}")


<details><summary>解答003</summary>

``````python
# 解答003

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]

# --- ノード定義 (Nodes) ---
def uppercase_node(state: GraphState):
    # 最新のメッセージを大文字に変換するノード
    last_message_content = state["messages"][-1].content
    return {"messages": [AIMessage(content=last_message_content.upper())]}

def reverse_node(state: GraphState):
    # 最新のメッセージを逆順にするノード
    last_message_content = state["messages"][-1].content
    return {"messages": [AIMessage(content=last_message_content[::-1])]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(GraphState)

# ノードの追加
workflow.add_node("uppercase", uppercase_node)
workflow.add_node("reverse", reverse_node)

# エントリポイントの設定
workflow.set_entry_point("uppercase")

# エッジの追加 (直列接続)
workflow.add_edge("uppercase", "reverse")
workflow.add_edge("reverse", END)

# グラフのコンパイル
graph = workflow.compile()

# --- グラフの実行と結果表示 ---
inputs = {"messages": [HumanMessage(content="Hello LangGraph")]}

final_state = graph.invoke(inputs)
print(f"最終的な応答: {final_state['messages'][-1].content}")
``````
</details>


<details><summary>解説003</summary>

#### この問題のポイント

*   **コード解説:**
    *   `uppercase_node`と`reverse_node`は、それぞれ入力メッセージを大文字化、逆順化する処理を行います。重要なのは、各ノードが新しい`AIMessage`を作成して状態に返す点です。これにより、次のノードは前のノードの処理結果を`state["messages"][-1]`で取得できます。
    *   `workflow.add_edge("uppercase", "reverse")`は、`uppercase`ノードの実行が完了したら、次に`reverse`ノードを実行するように指示します。このようにして、処理の流れを定義します。
    *   入力メッセージを`HumanMessage`として渡すことで、ユーザーからの入力であることを明示しています。ノードからの出力は`AIMessage`として返され、メッセージ履歴にAIの応答として記録されます。
---
</details>


### ■ 問題004: グラフ内でのLLMの利用（シンプルなチャットボット）

LangGraphのノード内で大規模言語モデル（LLM）を呼び出す方法を学び、シンプルなチャットボットを構築しましょう。ここでは、ユーザーからの入力に対してLLMが応答を生成し、その応答を返すグラフを作成します。


In [ ]:
# 解答欄004
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import ____, AIMessage
import os

# ノートブック冒頭で`llm`変数が初期化されている前提
# (from langchain_openai import ChatOpenAI や llm = ChatOpenAI(...) といった行はここには不要)

# --- 状態定義 (State) ---
class GraphState(____):
    messages: Annotated[list, add_messages]

# --- ノード定義 (Nodes) ---
def llm_node(state: GraphState):
    # LLMを呼び出し、応答を生成するノード
    # ノートブック冒頭で初期化された共通の `llm` 変数を使用します。
    response = ____.invoke(state["messages"])
    return {"messages": [response]} # responseはAIMessageオブジェクトを期待 (ここは歯抜けにしない)

# --- グラフ構築 (Graph) ---
workflow = ____(GraphState)

# ノードの追加
workflow.add_node("llm_responder", llm_node)

# エントリポイントの設定
workflow.____("llm_responder")

# 終了ポイントの設定
workflow.add_edge("llm_responder", ____)

# グラフのコンパイル
graph = workflow.compile()


In [ ]:
# --- グラフの可視化 ---
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")


In [ ]:
# --- グラフの実行と結果表示 ---
print("\n--- チャットボットのテスト ---")
# 最初のメッセージはHumanMessageであると想定
inputs = {"messages": [HumanMessage(content="こんにちは、あなたの名前は何ですか？")]}

final_state = graph.invoke(inputs)
print(f"最終的な応答: {final_state['messages'][-1].content}")

print("\n--- 別の質問 ---")
inputs2 = {"messages": [HumanMessage(content="今日の天気は？")]}

final_state2 = graph.invoke(inputs2)
print(f"最終的な応答: {final_state2['messages'][-1].content}")


<details><summary>解答004</summary>

``````python
# 解答004

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
import os # osはAPIキー設定のコメントアウト部分で使われているので残しても良いが、直接は不要になる

# ノートブック冒頭で`llm`変数が初期化されている前提

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]

# --- ノード定義 (Nodes) ---
def llm_node(state: GraphState):
    # LLMを呼び出し、応答を生成するノード
    # ノートブック冒頭で初期化された共通の `llm` 変数を使用します。
    response = llm.invoke(state["messages"]) # 共通llmを使用
    return {"messages": [response]} # responseはAIMessageオブジェクトを期待

# --- グラフ構築 (Graph) ---
workflow = StateGraph(GraphState)

# ノードの追加
workflow.add_node("llm_responder", llm_node)

# エントリポイントの設定
workflow.set_entry_point("llm_responder")

# 終了ポイントの設定
workflow.add_edge("llm_responder", END)

# グラフのコンパイル
graph = workflow.compile()

# --- グラフの実行と結果表示 ---
print("\n--- チャットボットのテスト ---")
# 最初のメッセージはHumanMessageであると想定
inputs = {"messages": [HumanMessage(content="こんにちは、あなたの名前は何ですか？")]}

final_state = graph.invoke(inputs)
print(f"最終的な応答: {final_state['messages'][-1].content}")

print("\n--- 別の質問 ---")
inputs2 = {"messages": [HumanMessage(content="今日の天気は？")]}

final_state2 = graph.invoke(inputs2)
print(f"最終的な応答: {final_state2['messages'][-1].content}")
``````
</details>


<details><summary>解説004</summary>

このノートブックでは、様々なLLMプラットフォーム（OpenAI, Azure OpenAI, Google Cloud Vertex AI, Google Gemini (Gemini API), Anthropic Claude, Amazon Bedrockなど）を簡単に切り替えて試せるように設計されています。
ノートブックの冒頭にある `LLM_PROVIDER` 変数で使用したいLLMを選択し、対応するAPIキーや環境変数を設定するだけで、この問題を含む全てのLLM呼び出し箇所で選択したLLMが利用されます。
選択した `LLM_PROVIDER` に応じて、必要なAPIキーが設定されているか（環境変数またはGoogle Colabのシークレット経由）、ノートブック起動時にチェックされます。

ここでは、ノートブックの先頭で設定・初期化された共通の `llm` 変数を使用して、LLMに質問をしています。
`llm.invoke()` という統一されたインターフェースで、どのLLMプロバイダーを利用しているかに関わらず、同じようにLLMを呼び出すことができます。
これにより、特定のLLMサービスに依存しない、より汎用的なコードを作成するメリットを手軽に体験できます。

もしエラーが発生した場合は、ノートブック冒頭の `LLM_PROVIDER` の設定、および選択したプロバイダーに応じたAPIキーや環境変数の設定（例: `OPENAI_API_KEY`, `GOOGLE_API_KEY`, `AZURE_OPENAI_ENDPOINT`など）が正しく行われているかを確認してください。
各プロバイダー固有の設定項目（例えばVertex AIのProject ID、AzureのDeployment Name、Bedrockのリージョンなど）も見直してください。
プロバイダーによっては、`pip install` コマンドで対応するライブラリ (例: `langchain-google-genai`) がインストールされているかも確認点です。

</details>


### ■ 問題005: 状態の更新 - 特定キーの値を上書きする

`add_messages` によるメッセージ履歴の追加だけでなく、グラフの状態(`State`)内の特定のキーの値を直接更新する方法を学びましょう。ここでは、カウンター値を保持する状態キーを定義し、ノードでその値をインクリメントするグラフを作成します。

*   **学習内容:** `TypedDict`で定義する状態クラスに、`messages`以外のカスタムキー（ここでは`counter: int`）を追加し、ノード関数内でその値を直接読み書きする方法を学びます。これにより、メッセージ履歴だけでなく、数値や文字列、ブール値など、より多様なデータをグラフ全体で管理できるようになります。


In [ ]:
# 解答欄005

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, ____
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage

# --- 状態定義 (State) ---
class CounterState(____):
    messages: Annotated[list, add_messages]
    counter: ____ 

# --- ノード定義 (Nodes) ---
def increment_counter(state: CounterState):
    # counterの値を1増やすノード
    current_count = state.get("counter", 0) # stateからcounterの値を取得、なければ0
    new_count = current_count + 1
    return {____: new_count, "messages": [HumanMessage(content=f"Counter incremented to {new_count}")]}

def display_count(state: CounterState):
    # counterの最終値を表示するノード (実際にはmessagesに追加されたもので確認)
    # このノードは状態を更新しないが、メッセージを追加しても良い
    return {"messages": [HumanMessage(content=f"Final count: {state['counter']}")]}

# --- グラフ構築 (Graph) ---
workflow = ____(CounterState)

# ノードの追加
workflow.add_node("increment", increment_counter)
workflow.add_node("display", display_count)

# エントリポイントの設定
workflow.set_entry_point("increment")

# エッジの追加
workflow.add_edge("increment", "display")
workflow.add_edge("display", ____)

# グラフのコンパイル
graph = workflow.compile()


In [ ]:
# --- グラフの可視化 ---
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")


In [ ]:
# --- グラフの実行と結果表示 ---
print("\n--- カウンターテスト (初期値0から) ---")
inputs = {"messages": [HumanMessage(content="Start counting")], "counter": 0} # 初期カウンター値を設定

final_state = graph.invoke(inputs)
print(f"最終的な応答: {final_state['messages'][-1].content}")

print("\n--- カウンターテスト (初期値5から) ---")
inputs_2 = {"messages": [HumanMessage(content="Start counting from 5")], "counter": 5} # 初期カウンター値を設定

final_state_2 = graph.invoke(inputs_2)
print(f"最終的な応答: {final_state_2['messages'][-1].content}")


<details><summary>解答005</summary>

``````python
# 解答005

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage

# --- 状態定義 (State) ---
class CounterState(TypedDict):
    messages: Annotated[list, add_messages]
    counter: int # 新しくカウンター用の状態キーを定義

# --- ノード定義 (Nodes) ---
def increment_counter(state: CounterState):
    # counterの値を1増やすノード
    current_count = state.get("counter", 0) # stateからcounterの値を取得、なければ0
    new_count = current_count + 1
    return {"counter": new_count, "messages": [HumanMessage(content=f"Counter incremented to {new_count}")]}

def display_count(state: CounterState):
    # counterの最終値を表示するノード (実際にはmessagesに追加されたもので確認)
    # このノードは状態を更新しないが、メッセージを追加しても良い
    return {"messages": [HumanMessage(content=f"Final count: {state['counter']}")]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(CounterState)

# ノードの追加
workflow.add_node("increment", increment_counter)
workflow.add_node("display", display_count)

# エントリポイントの設定
workflow.set_entry_point("increment")

# エッジの追加
workflow.add_edge("increment", "display")
workflow.add_edge("display", END)

# グラフのコンパイル
graph = workflow.compile()

# --- グラフの実行と結果表示 ---
print("\n--- カウンターテスト (初期値0から) ---")
inputs = {"messages": [HumanMessage(content="Start counting")], "counter": 0} # 初期カウンター値を設定

final_state = graph.invoke(inputs)
print(f"最終的な応答: {final_state['messages'][-1].content}")

print("\n--- カウンターテスト (初期値5から) ---")
inputs_2 = {"messages": [HumanMessage(content="Start counting from 5")], "counter": 5} # 初期カウンター値を設定

final_state_2 = graph.invoke(inputs_2)
print(f"最終的な応答: {final_state_2['messages'][-1].content}")
``````
</details>


<details><summary>解説005</summary>

#### この問題のポイント

*   **コード解説:**
    *   `CounterState`に`counter: int`を追加しました。これにより、グラフの状態はメッセージリストと整数型のカウンターを持つことになります。
    *   `increment_counter`ノードでは、`state.get("counter", 0)`を使って現在のカウンター値を取得しています。`.get()`メソッドを使うことで、キーが存在しない場合のデフォルト値を指定できます（ここでは初回実行時を想定して0）。その後、値をインクリメントし、更新後の値を`{"counter": new_count}`という辞書形式で返しています。LangGraphは、ノードが返す辞書のキーに基づいて対応する状態を更新します。
    *   `messages`キーも同時に返すことで、状態更新のログや情報をメッセージ履歴に残すことができます。
    *   グラフ実行時 (`graph.invoke`や`graph.stream`) に、`inputs`辞書に`"counter": 0`（または任意の値）を含めることで、`counter`キーの初期値を設定できます。
    *   このように、ノードは状態の一部または全部を更新する辞書を返すことで、グラフの状態を変化させます。`add_messages`はメッセージリストの更新に特化した便利な方法ですが、他のキーは直接値を指定して更新します。
---
</details>


### ■ 問題006: 状態の更新 - 複数のキーを一度に更新する

一つのノードから、状態(`State`)の複数のキーを同時に更新する方法を学びましょう。ここでは、ユーザーからのメッセージ内容に応じて、応答メッセージと共に「応答タイプ」という別の状態キーも更新するグラフを作成します。

*   **学習内容:** 一つのノード関数から返す辞書に複数のキーと値のペアを含めることで、グラフの状態(`State`)の複数の属性を一度に更新する方法を学びます。また、`typing.Literal`を使って、状態キーが取りうる値を制限する方法も示します。


In [ ]:
# 解答欄006
from typing import TypedDict, Annotated, Literal
from langgraph.graph import StateGraph, ____
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, ____

# --- 状態定義 (State) ---
ResponseType = Literal["greeting", "question", "other", "unknown"]

class MultiUpdateState(____):
    messages: Annotated[list, add_messages]
    response_type: ____
    last_user_message: str # 最後に入力されたユーザーメッセージ

# --- ノード定義 (Nodes) ---
def process_input(state: MultiUpdateState):
    user_message = state["messages"][-1].content.lower()
    response_text = ""
    resp_type: ResponseType = "unknown"

    if "こんにちは" in user_message or "こんばんは" in user_message:
        response_text = "こんにちは！何かお手伝いできますか？"
        resp_type = "greeting" 
    elif "?" in user_message or "教えて" in user_message:
        response_text = "ご質問ありがとうございます。それについては現在調べています。"
        resp_type = "question" 
    else:
        response_text = "メッセージありがとうございます。"
        resp_type = "other" 
    
    # 複数のキーを同時に更新して返す
    return {
        "messages": [AIMessage(content=response_text)],
        "response_type": resp_type,
        ____: user_message
    }

# --- グラフ構築 (Graph) ---
workflow = StateGraph(MultiUpdateState)

workflow.add_node("processor", process_input)
workflow.set_entry_point("processor")
workflow.add_edge("processor", ____)

graph = workflow.compile()


In [ ]:
# --- グラフの可視化 ---
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")


In [ ]:
# --- グラフの実行と結果表示 ---
test_inputs = [
    {"messages": [HumanMessage(content="こんにちは")]},
    {"messages": [HumanMessage(content="LangGraphについて教えてください")]},
    {"messages": [HumanMessage(content="今日の天気は晴れですね")]}
]

for i, inputs in enumerate(test_inputs):
    print(f"\n--- テスト実行 {i+1} ---")

    final_state = graph.invoke(inputs, {"recursion_limit": 3})
    print(f"最終的な応答: {final_state['messages'][-1].content}")
    assert "response_type" in final_state
    assert "last_user_message" in final_state
    print(f"Response Type: {final_state['response_type']}")
    print(f"Last User Message: {final_state['last_user_message']}")


<details><summary>解答006</summary>

``````python
# 解答006
from typing import TypedDict, Annotated, Literal
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

# --- 状態定義 (State) ---
ResponseType = Literal["greeting", "question", "other", "unknown"]

class MultiUpdateState(TypedDict):
    messages: Annotated[list, add_messages]
    response_type: ResponseType # 応答の種類を保持するキー
    last_user_message: str # 最後に入力されたユーザーメッセージ

# --- ノード定義 (Nodes) ---
def process_input(state: MultiUpdateState):
    user_message = state["messages"][-1].content.lower()
    response_text = ""
    resp_type: ResponseType = "unknown"

    if "こんにちは" in user_message or "こんばんは" in user_message:
        response_text = "こんにちは！何かお手伝いできますか？"
        resp_type = "greeting"
    elif "?" in user_message or "教えて" in user_message:
        response_text = "ご質問ありがとうございます。それについては現在調べています。"
        resp_type = "question"
    else:
        response_text = "メッセージありがとうございます。"
        resp_type = "other"
    
    # 複数のキーを同時に更新して返す
    return {
        "messages": [AIMessage(content=response_text)],
        "response_type": resp_type,
        "last_user_message": user_message # 元のユーザーメッセージを保存
    }

# --- グラフ構築 (Graph) ---
workflow = StateGraph(MultiUpdateState)

workflow.add_node("processor", process_input)
workflow.set_entry_point("processor")
workflow.add_edge("processor", END)

graph = workflow.compile()

# --- グラフの実行と結果表示 ---
test_inputs = [
    {"messages": [HumanMessage(content="こんにちは")]},
    {"messages": [HumanMessage(content="LangGraphについて教えてください")]},
    {"messages": [HumanMessage(content="今日の天気は晴れですね")]}
]

for i, inputs in enumerate(test_inputs):
    print(f"\n--- テスト実行 {i+1} ---")
    # 初期状態としてresponse_typeやlast_user_messageを渡すことも可能だが、
    # この問題ではノード内でこれらが設定されることを確認する
    initial_state = inputs.copy()
    # 必要であれば、初期値を設定
    # initial_state.setdefault("response_type", "unknown") 
    # initial_state.setdefault("last_user_message", "")

    final_state = graph.invoke(initial_state, {"recursion_limit": 3})
    print(f"最終的な応答: {final_state['messages'][-1].content}")
    assert "response_type" in final_state
    assert "last_user_message" in final_state
    print(f"Response Type: {final_state['response_type']}")
    print(f"Last User Message: {final_state['last_user_message']}")
``````
</details>


<details><summary>解説006</summary>

#### この問題のポイント

*   **コード解説:**
    *   `MultiUpdateState`に、`response_type: ResponseType` と `last_user_message: str` という2つの新しいキーを追加しました。`ResponseType`は`Literal["greeting", "question", "other", "unknown"]`で定義され、`response_type`キーがこれらの文字列のうちのいずれかの値を取ることを示します（型ヒントであり、実行時の厳密な強制ではありませんが、開発時の可読性や静的解析に役立ちます）。
    *   `process_input`ノードは、ユーザーのメッセージ内容に基づいて応答メッセージを生成し、それと同時に`response_type`（挨拶、質問、その他など）と`last_user_message`（処理対象となった元のユーザーメッセージ）も決定します。
    *   ノードが返す辞書は `{"messages": ..., "response_type": ..., "last_user_message": ...}` のようになります。LangGraphは、この辞書に含まれる各キーに対応する状態を更新します。
    *   実行時には、`messages`キーだけでなく、`response_type`と`last_user_message`も最終状態に含まれていることを確認できます。
    *   このように、ノードはグラフの状態を柔軟に更新する役割を担います。返す辞書に含めるキーと値によって、どの状態属性をどのように変更するかを制御できます。
---
</details>


### ■ 問題007: `END` 以外の終了ノードの指定（概念理解）

LangGraphでは、グラフの終点は通常、特別な `END` ノードに接続することで示されます。しかし、概念的には、あるノードが処理の最終ステップであり、それ以上後続のノードが存在しない場合、そのノードが事実上の「終了ノード」として機能すると考えることもできます。この問題では、特定のノードを実行した後、明示的に `END` に接続せず、グラフがそこで停止することを確認します。ただし、LangGraphのベストプラクティスとしては、可能な限り `END` を使用することが推奨されます。


In [ ]:
# 解答欄007
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, ____

# --- 状態定義 (State) ---
class FinalNodeState(____):
    messages: ____[list, add_messages]
    status: str

# --- ノード定義 (Nodes) ---
def start_process(state: FinalNodeState):
    return {"status": "Processing", "messages": [AIMessage(content="Process initiated.")]}

def final_processing_node(state: FinalNodeState):
    # このノードが処理の最後とする
    final_message = "Process completed at final_processing_node."
    return {"status": "Completed", "messages": [AIMessage(content=final_message)]}

# --- グラフ構築 (Graph) ---
workflow = ____(FinalNodeState)

workflow.add_node("start", start_process)
workflow.add_node("final_step", final_processing_node)

workflow.set_entry_point("start")

# startノードからfinal_stepノードへのエッジ
workflow.add_edge("start", ____)

# final_stepノードからENDへのエッジを意図的に作成しない
# workflow.____("final_step", END) # ← これをコメントアウトまたは削除

# グラフのコンパイル
# check_interruptions=True をつけると、ENDに到達しない場合にエラーになるため、
# この例では明示的にENDに繋がないことを示すために compile() の引数なし、
# または check_interruptions=False (デフォルト) を利用します。
# Langfuseなどのトレーシングツールと連携する場合、ENDに到達しないとトレースが終了しないことがあるため注意。
graph = workflow.compile()


In [ ]:
# --- グラフの可視化 ---
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")


In [ ]:
# --- グラフの実行と結果表示 ---
print("\n--- 最終ノードテスト ---")
inputs = {"messages": [HumanMessage(content="Begin process")], "status": "Initial"}
final_state_from_stream = None

print("Streaming execution:")
for chunk in graph.stream(inputs, {"recursion_limit": 5}):
    print(f"  Stream chunk: {chunk}")
    final_state_from_stream = chunk # Capture the last chunk which contains the state of the last executed node

print(f"Final State from stream: {final_state_from_stream}")

# invokeの挙動確認
invoked_state = None
try:
    invoked_state = graph.invoke(inputs, {"recursion_limit": 5})
    print(f"最終的な応答 (invoke): {invoked_state['messages'][-1].content}")
except Exception as e:
    print(f"Invoke call resulted in an error or unexpected behavior: {e}")
    print("This might be expected if the graph doesn't explicitly reach END.")

assert final_state_from_stream is not None, "Final state was not captured from stream"
final_node_key = list(final_state_from_stream.keys())[0] # Get the key of the last node's state
assert final_state_from_stream[final_node_key]["status"] == "Completed"
assert "Process completed at final_processing_node." in final_state_from_stream[final_node_key]["messages"][-1].content
print("Assertion for final_state_from_stream passed.")

if invoked_state:
    print(f"Invoked state status: {invoked_state.get('status')}")

<details><summary>解答007</summary>

``````python
# 解答007
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from IPython.display import Image, display

# --- 状態定義 (State) ---
class FinalNodeState(TypedDict):
    messages: Annotated[list, add_messages]
    status: str

# --- ノード定義 (Nodes) ---
def start_process(state: FinalNodeState):
    return {"status": "Processing", "messages": [AIMessage(content="Process initiated.")]}

def final_processing_node(state: FinalNodeState):
    # このノードが処理の最後とする
    final_message = "Process completed at final_processing_node."
    return {"status": "Completed", "messages": [AIMessage(content=final_message)]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(FinalNodeState)

workflow.add_node("start", start_process)
workflow.add_node("final_step", final_processing_node)

workflow.set_entry_point("start")

# startノードからfinal_stepノードへのエッジ
workflow.add_edge("start", "final_step")

# final_stepノードからENDへのエッジを意図的に作成しない
# workflow.add_edge("final_step", END) # ← コメントアウトまたは削除

# グラフのコンパイル
graph = workflow.compile()

# --- グラフの実行と結果表示 ---
print("\n--- 最終ノードテスト ---")
inputs = {"messages": [HumanMessage(content="Begin process")], "status": "Initial"}
final_state_from_stream = None

print("Streaming execution:")
for chunk in graph.stream(inputs, {"recursion_limit": 5}):
    print(f"  Stream chunk: {chunk}")
    final_state_from_stream = chunk

print(f"Final State from stream: {final_state_from_stream}")

invoked_state = None
try:
    invoked_state = graph.invoke(inputs, {"recursion_limit": 5})
    print(f"最終的な応答 (invoke): {invoked_state['messages'][-1].content}")
except Exception as e:
    print(f"Invoke call resulted in an error or unexpected behavior: {e}")
    print("This might be expected if the graph doesn't explicitly reach END.")

assert final_state_from_stream is not None, "Final state was not captured from stream"
final_node_key = list(final_state_from_stream.keys())[0]
assert final_state_from_stream[final_node_key]["status"] == "Completed"
assert "Process completed at final_processing_node." in final_state_from_stream[final_node_key]["messages"][-1].content
print("Assertion for final_state_from_stream passed.")

if invoked_state:
    print(f"Invoked state status: {invoked_state.get('status')}")

``````
</details>


<details><summary>解説007</summary>

#### この問題のポイント
*   **学習内容:** グラフの終点として必ずしも `END` を明示的に指定する必要はなく、あるノードから先に遷移するエッジがなければ、そのノードの処理が終わった時点でグラフの実行が停止し、その時点での状態が最終状態となることを理解します。ただし、これはLangGraphの挙動の一つであり、デバッグや可視化、他のツールとの連携（例: Langfuse）を考慮すると、可能な限りグラフの終点を `END` に接続することが推奨されます。
*   **コード解説:**
    *   `final_processing_node`を作成し、このノードから `END` へのエッジ（`workflow.add_edge("final_step", END)`）を定義していません。
    *   `graph.stream()` を使ってグラフを実行すると、`final_processing_node` が実行された後、それ以上進むべきノードがないため、処理が停止します。`stream()` の最後の出力（この場合は `final_processing_node` の出力）が、その実行における最終的な状態を示します。
    *   `graph.invoke()` の場合、グラフが明示的に `END` に到達しないと、バージョンや設定によってはエラーが発生したり、予期しない挙動をしたりする可能性があります。一般的に `invoke()` はグラフが `END` に到達し、完全な最終状態が確定することを期待します。この問題では、主に `stream()` での挙動を確認し、`invoke()` は参考として示しています。
    *   可視化すると、`final_step` ノードから `END` (または他のノード) への矢印がないことが確認できます。
*   **重要な注意点:**
    *   **`END` の使用推奨:** LangGraphでは、グラフの論理的な終了点を明確にするために `END` を使用することが強く推奨されます。これにより、グラフの構造が理解しやすくなり、デバッグも容易になります。また、LangSmith/Langfuseのようなトレースツールは、`END` への到達をもって一連の処理の完了とみなすことが多いため、連携時にも重要です。
    *   **`compile(check_interruptions=True)`:** グラフをコンパイルする際に `check_interruptions=True` を指定すると、中断（Interrupt）が発生しない限り、グラフが必ず `END` に到達することを強制できます。`END` に到達しないパスがある場合、コンパイル時または実行時にエラーが発生します。
    *   この問題は、`END` を使わない場合の挙動を理解するためのものであり、実際の開発では `END` を適切に配置する設計を心がけてください。
---
</details>


### ■ 問題008: LLMノードと非LLMノードの連携強化

LLM（大規模言語モデル）を組み込んだノードと、LLM以外の処理を行うノード（例: 文字列操作、データ抽出など）を連携させる方法を学びましょう。ここでは、LLMに質問を投げて得られた応答（文字列）から、特定の情報を抽出・加工して状態を更新する、より実践的なグラフを作成します。

*   **学習内容:** LLMを呼び出すノードと、その出力を処理する非LLMノードを組み合わせることで、より高度な情報処理パイプラインを構築する方法を学びます。具体的には、LLMの自然言語応答から正規表現などを用いて構造化された情報を抽出し、グラフの状態を更新します。


In [ ]:
# 解答欄008
from typing import TypedDict, Annotated
import re # 正規表現モジュールをインポート
from langgraph.graph import StateGraph, ____
from langgraph.graph.message import add_messages
from langchain_core.messages import ____, AIMessage

# ノートブック冒頭で`llm`変数が初期化されている前提

# --- 状態定義 (State) ---
class ExtractionState(____):
    messages: ____[list, add_messages]
    user_question: str # ユーザーの元の質問
    llm_response_text: str # LLMの生の応答テキスト
    extracted_info: str | None # LLMの応答から抽出された情報

# --- ノード定義 (Nodes) ---
def get_user_question(state: ExtractionState):
    # ユーザーの質問を状態に保存
    last_message_content = state["messages"][-1].content
    return {"user_question": last_message_content}

def llm_responder_node(state: ExtractionState):
    # LLMに質問を投げるノード
    question = state["user_question"]
    # LLMに渡すメッセージは、過去の履歴全体でも、最新の質問だけでも良い
    # ここでは簡単のため、最新の質問のみをHumanMessageとして渡す
    response = llm.invoke([HumanMessage(content=question)])
    response_content = response.content
    return {"messages": [response], "llm_response_text": response_content}

def data_extractor_node(state: ExtractionState):
    # LLMの応答から情報を抽出するノード
    raw_response = state["llm_response_text"]
    # 例: LLMが「日本の首都は東京です。」と答えたら「東京」を抽出
    # ここでは簡単な正規表現で「XXはYYです」のYY部分を抽出試行
    extracted = None
    match = re.search(r"(?:は|is)\s*([^。.]+)[.。]?", raw_response) # 簡易的な抽出
    if match:
        extracted = match.group(1).strip()
    
    return {____: extracted, "messages": [AIMessage(content=f"Extracted: {extracted}")]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(ExtractionState)

workflow.add_node("capture_question", get_user_question)
workflow.add_node("ask_llm", llm_responder_node)
workflow.add_node("extract_data", data_extractor_node)

workflow.set_entry_point("capture_question")

workflow.add_edge("capture_question", "ask_llm")
workflow.add_edge("ask_llm", ____)
workflow.add_edge("extract_data", ____)

graph = workflow.compile()


In [ ]:
# --- グラフの可視化 ---
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")


In [ ]:
# --- グラフの実行と結果表示 ---
questions = [
    "日本の首都は何ですか？",
    "フランスの有名な画家の名前を一人教えてください。", # LLMの回答次第で抽出成功/失敗が変わる
    "1+1は何？", # LLMが「2です」と答えれば抽出できるかも
    "今日の天気は？" # 「晴れです」なら「晴れ」を抽出期待
]

for q_text in questions:
    print(f"\n--- LLM連携と情報抽出テスト (質問: {q_text}) ---")
    inputs = {"messages": [HumanMessage(content=q_text)]}

    final_state_invoked = graph.invoke(inputs, {"recursion_limit": 5})
    print(f"最終的な応答: {final_state_invoked['messages'][-1].content}")
    print(f"  User Question: {final_state_invoked.get('user_question')}")
    print(f"  LLM Response: {final_state_invoked.get('llm_response_text')}")
    print(f"  Extracted Info: {final_state_invoked.get('extracted_info')}")

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"Graph visualization failed: {e}")


<details><summary>解答008</summary>

``````python
# 解答008
from typing import TypedDict, Annotated
import re # 正規表現モジュールをインポート
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from IPython.display import Image, display

# ノートブック冒頭で`llm`変数が初期化されている前提

# --- 状態定義 (State) ---
class ExtractionState(TypedDict):
    messages: Annotated[list, add_messages]
    user_question: str # ユーザーの元の質問
    llm_response_text: str # LLMの生の応答テキスト
    extracted_info: str | None # LLMの応答から抽出された情報

# --- ノード定義 (Nodes) ---
def get_user_question(state: ExtractionState):
    # ユーザーの質問を状態に保存
    last_message_content = state["messages"][-1].content
    return {"user_question": last_message_content}

def llm_responder_node(state: ExtractionState):
    # LLMに質問を投げるノード
    question = state["user_question"]
    response = llm.invoke([HumanMessage(content=question)])
    response_content = response.content
    return {"messages": [response], "llm_response_text": response_content}

def data_extractor_node(state: ExtractionState):
    # LLMの応答から情報を抽出するノード
    raw_response = state["llm_response_text"]
    extracted = None
    # 改善された正規表現: 「XXはYYです」「XX is YY」のようなパターンや、単に「YYです」のような応答にも対応試行
    # 質問が「日本の首都は何ですか？」で応答が「東京です。」の場合「東京」を抽出
    # 質問が「日本の首都は？」で応答が「東京」の場合「東京」を抽出
    patterns = [
        r"(?:.+は|.+\s*is)\s*(.+?)(?:です|。|\.|\s*for|$)", #「～はXです」「～ is X」
        r"^([^。.]+?)(?:です|。|\.|\s*for|$)" # 文頭から「Xです」
    ]
    for pattern in patterns:
        match = re.search(pattern, raw_response)
        if match:
            extracted = match.group(1).strip()
            if extracted.lower() == state["user_question"].lower().replace("何ですか","").replace("何","зиру").strip("?？") : # 質問自体が答えになるような場合を除外
                 extracted = None # 例：「天気は？」->「晴れです」はOKだが、「天気は？」->「天気」はNG
                 continue
            break
    
    # もし上記で抽出できなかった場合、LLMの応答が単語やフレーズそのものである可能性を考慮
    if not extracted and len(raw_response.split()) < 5 and not state["user_question"].startswith(raw_response): # 短い応答で、質問の繰り返しでない場合
        extracted = raw_response.strip()

    return {"extracted_info": extracted, "messages": [AIMessage(content=f"Extracted info: {extracted if extracted else 'N/A'}")]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(ExtractionState)

workflow.add_node("capture_question", get_user_question)
workflow.add_node("ask_llm", llm_responder_node)
workflow.add_node("extract_data", data_extractor_node)

workflow.set_entry_point("capture_question")

workflow.add_edge("capture_question", "ask_llm")
workflow.add_edge("ask_llm", "extract_data")
workflow.add_edge("extract_data", END)

graph = workflow.compile()

# --- グラフの実行と結果表示 ---
questions = [
    "日本の首都は何ですか？",
    "フランスの有名な画家の名前を一人教えてください。", # LLMの回答次第で抽出成功/失敗が変わる
    "1+1は何？", # LLMが「2です」と答えれば抽出できるかも
    "今日の天気は？" # 「晴れです」なら「晴れ」を抽出期待
]

for q_text in questions:
    print(f"\n--- LLM連携と情報抽出テスト (質問: {q_text}) ---")
    inputs = {"messages": [HumanMessage(content=q_text)], "llm_response_text": "", "extracted_info": None} # 初期値を設定
    
    final_state_data = graph.invoke(inputs, {"recursion_limit": 5})

    if final_state_data:
        print(f"最終的な応答: {final_state_data['messages'][-1].content}")
        print(f"  User Question: {final_state_data.get('user_question')}")
        print(f"  LLM Response: {final_state_data.get('llm_response_text')}")
        print(f"  Extracted Info: {final_state_data.get('extracted_info')}")
    else:
        print("Could not retrieve final state.")
``````
</details>


<details><summary>解説008</summary>

#### この問題のポイント

*   **コード解説:**
    *   `ExtractionState`には、ユーザーの質問 (`user_question`)、LLMの生の応答テキスト (`llm_response_text`)、そして抽出された情報 (`extracted_info`) を保持するためのキーが定義されています。
    *   `get_user_question`ノード: ユーザーからの最初のメッセージを `user_question` として状態に保存します。
    *   `llm_responder_node`: 保存された `user_question` を使ってLLMに問い合わせを行い、得られた応答を `messages` (AIMessageとして) と `llm_response_text` (生の文字列として) 状態に保存します。
    *   `data_extractor_node`: `llm_response_text` から情報を抽出します。この例では、簡単な正規表現 `re.search(r"(?:は|is)\s*([^。.]+)[.。]?", raw_response)` を使用して、「AはBです」や「A is B」といった形式の文からBの部分を抽出しようと試みています。抽出結果は `extracted_info` として状態に保存されます。正規表現は完璧ではなく、LLMの応答形式によってはうまく抽出できない場合もありますが、ここではLLMの出力後処理の一例として示しています。
    *   グラフは `capture_question` -> `ask_llm` -> `extract_data` -> `END` というシーケンシャルな流れで定義されています。
    *   実行時には、異なる質問を投げ、LLMの応答とそこから抽出された情報（または抽出できなかった場合は `None` や `N/A`）が最終状態に含まれることを確認します。
*   **発展:**
    *   情報抽出の方法は正規表現に限らず、より高度なNLPライブラリ（例: spaCy）や、LangChainが提供するOutput Parsers、あるいは別のLLMコール（Function Calling/Tool Calling対応モデルならより高精度）を使って行うことも考えられます。
    *   抽出に失敗した場合のフォールバック処理（例: ユーザーに再確認を求める、デフォルト値を設定するなど）をグラフに追加することもできます。
---
</details>


### ■ 問題009: グラフの入力と出力のカスタマイズと明確化

LangGraphのグラフを実行する際、invoke() メソッドに渡す初期状態の構造と、グラフ全体の最終的な出力状態の構造を意識することが重要です。StateGraph に渡す状態クラス（例: TypedDict）の定義が、実質的にグラフの入力と出力のスキーマ（型定義）となります。この問題では、入力として複数の情報を受け取り、それらを複数のノードで段階的に処理して特定の構造で出力するグラフを作成し、入出力の対応関係を明確に意識します。

*   **学習内容:** `TypedDict` を使ってグラフの状態スキーマを定義する際、どのキーがグラフへの「入力」として期待され、どのキーが処理の「中間状態」として使われ、どのキーが最終的な「出力」として扱われるのかを明確に意識することを学びます。また、直線的なグラフ構造で状態がどのように更新されていくかを確認します。


In [ ]:
# 解答欄009
from typing import TypedDict, Annotated, List, Optional
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

# --- 状態定義 (入力、中間、出力のスキーマを兼ねる) ---
class ProcessedData(TypedDict):
    item_id: str
    description: str
    is_processed: bool

class ComplexIOState(____):
    # 入力として期待されるキー
    raw_item_name: str
    raw_item_details: List[str]

    # 処理中に使われるキー (中間状態)
    messages: Annotated[list, add_messages]
    enriched_description: Optional[str]

    # 出力として期待される主要なキー
    processed_data: Optional[____] # 処理結果
    error_message: Optional[str] # エラー発生時のメッセージ

# --- ノード定義 (Nodes) ---
def initialize_state(state: ComplexIOState):
    """入力値を元に、中間状態と出力状態を初期化するノード"""
    return {
        "messages": [AIMessage(content=f"Initializing for {state['raw_item_name']}")],
        "enriched_description": None,
        "processed_data": None,
        "error_message": None
    }

def enrich_description_node(state: ComplexIOState):
    """入力の詳細情報から、説明文を生成して状態を更新するノード"""
    details_text = ", ".join(state["raw_item_details"])
    if not details_text:
        # この時点ではエラーとせず、後続のノードで判断させる
        return {
            "messages": [AIMessage(content="No details provided to enrich.")],
        }
    
    description = f"This item has the following details: {details_text}."
    return {
        "messages": [AIMessage(content="Description enriched.")],
        "enriched_description": description
    }

def finalize_processing_node(state: ComplexIOState):
    """最終的な処理を行い、出力キーを確定させるノード"""
    name = state["raw_item_name"]
    
    # 前のノードで生成された説明文があるかチェック
    if not state.get("enriched_description"):
        err_msg = f"Failed to process {name}: No description was generated."
        return {
            "messages": [AIMessage(content=f"Error: {err_msg}")],
            "error_message": err_msg
        }

    # 成功した場合の処理
    processed_item = ProcessedData(
        item_id=f"PROC_{name.upper()}",
        description=state["enriched_description"],
        is_processed=True
    )
    return {
        "messages": [AIMessage(content=f"Successfully processed {name}")],
        "processed_data": processed_item
    }

# --- グラフ構築 (Graph) ---
workflow = StateGraph(ComplexIOState)

# ノードをグラフに追加
workflow.add_node("initializer", initialize_state)
workflow.add_node("enricher", enrich_description_node)
workflow.add_node("finalizer", ____)

# ノードをエッジでつなぐ
workflow.set_entry_point("initializer")
workflow.add_edge("initializer", "enricher")
workflow.add_edge(____, "finalizer")

# finalizerノードを終点として設定
workflow.add_edge("finalizer", ____)

graph = workflow.compile()

In [ ]:
# --- グラフの可視化 ---
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")


In [ ]:
# --- グラフの実行と結果表示 ---
inputs_success = {
    "raw_item_name": "Laptop",
    "raw_item_details": ["16GB RAM", "512GB SSD", "M3 Chip"],
}

inputs_fail_no_details = {
    "raw_item_name": "EmptyItem",
    "raw_item_details": [], 
}

test_cases = {
    "Success Case": inputs_success, 
    "Failure Case (No Details)": inputs_fail_no_details,
}

for case_name, inputs_data in test_cases.items():
    print(f"\n--- I/Oカスタマイズテスト: {case_name} ---")
    
    # グラフ実行時の入力は、状態スキーマに定義したキーを持つ辞書
    final_output_state = graph.invoke(inputs_data)
    
    print(f"最終的な応答: {final_output_state['messages'][-1].content}")

    # 最終状態から、出力として定義したキーを取り出して結果を確認
    if final_output_state.get("processed_data"):
        print(f"  Processed Item ID: {final_output_state['processed_data']['item_id']}")
        print(f"  Description: {final_output_state['processed_data']['description']}")
        print(f"  Processed: {final_output_state['processed_data']['is_processed']}")
    if final_output_state.get("error_message"):
        print(f"  Error: {final_output_state['error_message']}")

<details><summary>解答009</summary>

``````python
# 解答009
from typing import TypedDict, Annotated, List, Optional
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from IPython.display import Image, display

# --- 状態定義 (入力、中間、出力のスキーマを兼ねる) ---
class ProcessedData(TypedDict):
    item_id: str
    description: str
    is_processed: bool

class ComplexIOState(TypedDict):
    # 入力として期待されるキー
    raw_item_name: str
    raw_item_details: List[str]

    # 処理中に使われるキー (中間状態)
    messages: Annotated[list, add_messages]
    enriched_description: Optional[str]

    # 出力として期待される主要なキー
    processed_data: Optional[ProcessedData] # 処理結果
    error_message: Optional[str] # エラー発生時のメッセージ

# --- ノード定義 (Nodes) ---
def initialize_state(state: ComplexIOState):
    """入力値を元に、中間状態と出力状態を初期化するノード"""
    return {
        "messages": [AIMessage(content=f"Initializing for {state['raw_item_name']}")],
        "enriched_description": None,
        "processed_data": None,
        "error_message": None
    }

def enrich_description_node(state: ComplexIOState):
    """入力の詳細情報から、説明文を生成して状態を更新するノード"""
    details_text = ", ".join(state["raw_item_details"])
    if not details_text:
        # この時点ではエラーとせず、後続のノードで判断させる
        return {
            "messages": [AIMessage(content="No details provided to enrich.")],
        }
    
    description = f"This item has the following details: {details_text}."
    return {
        "messages": [AIMessage(content="Description enriched.")],
        "enriched_description": description
    }

def finalize_processing_node(state: ComplexIOState):
    """最終的な処理を行い、出力キーを確定させるノード"""
    name = state["raw_item_name"]
    
    # 前のノードで生成された説明文があるかチェック
    if not state.get("enriched_description"):
        err_msg = f"Failed to process {name}: No description was generated."
        return {
            "messages": [AIMessage(content=f"Error: {err_msg}")],
            "error_message": err_msg
        }

    # 成功した場合の処理
    processed_item = ProcessedData(
        item_id=f"PROC_{name.upper()}",
        description=state["enriched_description"],
        is_processed=True
    )
    return {
        "messages": [AIMessage(content=f"Successfully processed {name}")],
        "processed_data": processed_item
    }

# --- グラフ構築 (Graph) ---
workflow = StateGraph(ComplexIOState)

# ノードをグラフに追加
workflow.add_node("initializer", initialize_state)
workflow.add_node("enricher", enrich_description_node)
workflow.add_node("finalizer", finalize_processing_node)

# ノードをエッジでつなぐ
workflow.set_entry_point("initializer")
workflow.add_edge("initializer", "enricher")
workflow.add_edge("enricher", "finalizer")

# finalizerノードを終点として設定
workflow.add_edge("finalizer", END)

graph = workflow.compile()

# --- グラフの可視化 ---
try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")

# --- グラフの実行と結果表示 ---
inputs_success = {
    "raw_item_name": "Laptop",
    "raw_item_details": ["16GB RAM", "512GB SSD", "M3 Chip"],
}

inputs_fail_no_details = {
    "raw_item_name": "EmptyItem",
    "raw_item_details": [], 
}

test_cases = {
    "Success Case": inputs_success, 
    "Failure Case (No Details)": inputs_fail_no_details,
}

for case_name, inputs_data in test_cases.items():
    print(f"\n--- I/Oカスタマイズテスト: {case_name} ---")
    
    # グラフ実行時の入力は、状態スキーマに定義したキーを持つ辞書
    final_output_state = graph.invoke(inputs_data)
    
    print(f"最終的な応答: {final_output_state['messages'][-1].content}")

    # 最終状態から、出力として定義したキーを取り出して結果を確認
    if final_output_state.get("processed_data"):
        print(f"  Processed Item ID: {final_output_state['processed_data']['item_id']}")
        print(f"  Description: {final_output_state['processed_data']['description']}")
        print(f"  Processed: {final_output_state['processed_data']['is_processed']}")
    if final_output_state.get("error_message"):
        print(f"  Error: {final_output_state['error_message']}")

``````
</details>


<details><summary>解説009</summary>

#### この問題のポイント

*   **コード解説:**
    *   **`ComplexIOState` (状態スキーマ):**
        *   **入力想定キー:** `raw_item_name`, `raw_item_details`。これらはグラフ実行時に `invoke` の `inputs` 引数で渡されることが期待される、グラフの「入力」です。
        *   **中間状態キー:** `messages`, `enriched_description`。これらはグラフ内部の処理で使われます。`enrich_description_node` が `enriched_description` を書き込み、`finalize_processing_node` がそれを読み取ります。このように、ノード間でデータを渡すために使われます。
        *   **出力想定キー:** `processed_data` (処理成功時の結果), `error_message` (エラー発生時の情報)。これらのキーの値が、グラフ実行後の最終的な成果物、つまりグラフの「出力」となります。
    *   **ノードの役割分担:**
        *   `initialize_state`: グラフの実行開始時に呼ばれ、中間状態や出力用のキーを `None` などで初期化します。
        *   `enrich_description_node`: 「入力」である `raw_item_details` を使って中間データ `enriched_description` を生成します。
        *   `finalize_processing_node`: 中間データ `enriched_description` を受け取り、それを元に最終的な「出力」である `processed_data` または `error_message` を生成します。
    *   **グラフの構造:**
        *   `set_entry_point()` で開始ノード (`initializer`) を指定します。
        *   `add_edge()` を使って、`initializer` → `enricher` → `finalizer` という直線的な処理の流れを定義します。
        *   最後の `workflow.add_edge("finalizer", END)` で、`finalizer` ノードの実行後にグラフが終了することを明示しています。(`END` は LangGraph からインポートした特別な定数です)
*   **重要な点:**
    *   状態スキーマ（`TypedDict`）は、グラフのインターフェース（入力と出力の形式）と、グラフ内部でのデータの受け渡し方法を定義する、中心的な役割を果たします。
    *   処理を複数のノードに分割し、それぞれが一つの責任を持つように設計することで、グラフの見通しが良くなります。
    *   ノードからノードへは、状態オブジェクト（この例では `ComplexIOState` のインスタンスである辞書）を介してデータが引き継がれます。前のノードが状態を更新し、次のノードがその更新された状態を読み取って処理を進めます。
---
</details>


### ■ 問題010: 複数のLLM呼び出しを含むグラフ

一つのグラフ内で、異なる役割やプロンプトを持つ複数のLLM呼び出しノードを組み込む方法を学びましょう。例えば、最初のLLMがアイデアを生成し、次のLLMがそのアイデアを評価・洗練する、といった連携が考えられます。この問題では、簡単な役割分担を持つ2つのLLMノードを直列に接続します。

*   **学習内容:** 一つのグラフ内に、それぞれ異なるプロンプトや役割を持つ複数のLLM呼び出しノードを配置し、それらを連携させる方法を学びます。これにより、より複雑で多段階の思考や処理を行うエージェントやパイプラインを構築できます。


In [ ]:
# 解答欄010
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, ____
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, ____
from langchain_core.prompts import ChatPromptTemplate

# ノートブック冒頭で`llm`変数が初期化されている前提

# --- 状態定義 (State) ---
class MultiLLMState(____):
    messages: Annotated[list, add_messages]
    original_topic: str # ユーザーからの最初のトピック
    generated_idea: str | None # アイデア生成LLMの出力
    evaluated_idea: str | None # アイデア評価LLMの出力

# --- ノード定義 (Nodes) ---
def get_topic(state: MultiLLMState):
    topic = state["messages"][-1].content
    return {"original_topic": topic, "messages": [AIMessage(content=f"Topic received: {topic}")]}

def idea_generation_node(state: MultiLLMState):
    topic = state["original_topic"]
    
    prompt_template_idea = ChatPromptTemplate.from_messages([
        ("system", "あなたは新しいアイデアを生み出すのが得意なAIです。与えられたトピックに関して、ユニークで面白いアイデアを一つ提案してください。アイデアは簡潔に一行で述べてください。"),
        ("human", "トピック: {topic}")
    ])
    
    chain = ____ | llm
    response = chain.invoke({"topic": topic})
    idea = response.content.strip()
    
    return {"generated_idea": idea, "messages": [AIMessage(content=f"Generated Idea: {idea}")]}

def idea_evaluation_node(state: MultiLLMState):
    idea = state["generated_idea"]
    if not idea:
        return {"messages": [AIMessage(content="No idea to evaluate.")], "evaluated_idea": "N/A"}
        
    
    prompt_template_eval = ChatPromptTemplate.from_messages([
        ("system", "あなたはアイデアを客観的に評価するのが得意なAIです。与えられたアイデアについて、その実現可能性と面白さを評価し、短いコメントを述べてください。"),
        ("human", "評価対象のアイデア: {idea}")
    ])
    
    chain = prompt_template_eval | llm
    response = chain.invoke({"idea": idea})
    evaluation = response.content.strip()
    
    return {"evaluated_idea": evaluation, "messages": [AIMessage(content=f"Evaluation: {evaluation}")]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(MultiLLMState)

workflow.add_node("capture_topic", get_topic)
workflow.add_node("generate_idea", idea_generation_node)
workflow.add_node("evaluate_idea", idea_evaluation_node)

workflow.set_entry_point("capture_topic")

workflow.add_edge("capture_topic", "generate_idea")
workflow.add_edge(____, "evaluate_idea")
workflow.add_edge("evaluate_idea", ____)

graph = workflow.compile()


In [ ]:
# --- グラフの可視化 ---
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")


In [ ]:
# --- グラフの実行と結果表示 ---
topics_to_test = [
    "新しい料理のレシピ",
    "未来の交通手段",
    "週末の過ごし方"
]

for topic_text in topics_to_test:
    print(f"\n--- 複数LLM連携テスト (トピック: {topic_text}) ---")
    inputs = {
        "messages": [HumanMessage(content=topic_text)],
        "original_topic": "", 
        "generated_idea": None,
        "evaluated_idea": None
    }
    final_state = graph.invoke(inputs, {"recursion_limit": 5})
    print(f"Original Topic: {final_state.get('original_topic')}")
    print(f"Generated Idea: {final_state.get('generated_idea')}")
    print(f"Evaluated Idea: \n{final_state.get('evaluated_idea')}")
    print("\n\n\n")


<details><summary>解答010</summary>

``````python
# 解答010
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from IPython.display import Image, display

# ノートブック冒頭で`llm`変数が初期化されている前提

# --- 状態定義 (State) ---
class MultiLLMState(TypedDict):
    messages: Annotated[list, add_messages]
    original_topic: str # ユーザーからの最初のトピック
    generated_idea: str | None # アイデア生成LLMの出力
    evaluated_idea: str | None # アイデア評価LLMの出力

# --- ノード定義 (Nodes) ---
def get_topic(state: MultiLLMState):
    topic = state["messages"][-1].content
    return {"original_topic": topic, "messages": [AIMessage(content=f"Topic received: {topic}")]}

def idea_generation_node(state: MultiLLMState):
    topic = state["original_topic"]
    
    prompt_template_idea = ChatPromptTemplate.from_messages([
        ("system", "あなたは新しいアイデアを生み出すのが得意なAIです。与えられたトピックに関して、ユニークで面白いアイデアを一つ提案してください。アイデアは簡潔に一行で述べてください。"),
        ("human", "トピック: {topic}")
    ])
    
    chain = prompt_template_idea | llm 
    response = chain.invoke({"topic": topic})
    idea = response.content.strip()
    
    return {"generated_idea": idea, "messages": [AIMessage(content=f"Generated Idea: {idea}")]}

def idea_evaluation_node(state: MultiLLMState):
    idea = state["generated_idea"]
    if not idea:
        return {"messages": [AIMessage(content="No idea to evaluate.")], "evaluated_idea": "N/A"}
        
    
    prompt_template_eval = ChatPromptTemplate.from_messages([
        ("system", "あなたはアイデアを客観的に評価するのが得意なAIです。与えられたアイデアについて、その実現可能性と面白さを評価し、短いコメントを述べてください。"),
        ("human", "評価対象のアイデア: {idea}")
    ])
    
    chain = prompt_template_eval | llm 
    response = chain.invoke({"idea": idea})
    evaluation = response.content.strip()
    
    return {"evaluated_idea": evaluation, "messages": [AIMessage(content=f"Evaluation: {evaluation}")]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(MultiLLMState)

workflow.add_node("capture_topic", get_topic)
workflow.add_node("generate_idea", idea_generation_node)
workflow.add_node("evaluate_idea", idea_evaluation_node)

workflow.set_entry_point("capture_topic")

workflow.add_edge("capture_topic", "generate_idea")
workflow.add_edge("generate_idea", "evaluate_idea")
workflow.add_edge("evaluate_idea", END)

graph = workflow.compile()

# --- グラフの実行と結果表示 ---
topics_to_test = [
    "新しい料理のレシピ",
    "未来の交通手段",
    "週末の過ごし方"
]

for topic_text in topics_to_test:
    print(f"\n--- 複数LLM連携テスト (トピック: {topic_text}) ---")
    inputs = {
        "messages": [HumanMessage(content=topic_text)],
        "original_topic": "", 
        "generated_idea": None,
        "evaluated_idea": None
    }
    final_state = graph.invoke(inputs, {"recursion_limit": 5})
    print(f"Original Topic: {final_state.get('original_topic')}")
    print(f"Generated Idea: {final_state.get('generated_idea')}")
    print(f"Evaluated Idea: \n{final_state.get('evaluated_idea')}")
    print("\n\n\n")
``````
</details>


<details><summary>解説010</summary>

#### この問題のポイント

*   **コード解説:**
    *   `MultiLLMState`には、ユーザーからの最初のトピック (`original_topic`)、最初のLLMが生成したアイデア (`generated_idea`)、そして二番目のLLMが評価した結果 (`evaluated_idea`) を保持するキーが定義されています。
    *   `get_topic`ノード: ユーザーの入力を `original_topic` として状態に保存します。
    *   `idea_generation_node`: `original_topic` に基づいて、アイデア生成用のプロンプト (`prompt_template_idea`) を使用してLLMを呼び出し、結果を `generated_idea` に保存します。
    *   `idea_evaluation_node`: `generated_idea` に基づいて、アイデア評価用のプロンプト (`prompt_template_eval`) を使用してLLMを呼び出し（ここでも同じ `llm` インスタンスを使用していますが、プロンプトが異なるため役割が変わります）、結果を `evaluated_idea` に保存します。
    *   グラフは `capture_topic` -> `generate_idea` -> `evaluate_idea` -> `END` という直列な流れで、各ステップで状態が更新されていきます。
    *   各LLM呼び出しノード内では、`langchain_core.prompts.ChatPromptTemplate` を使ってそのノード専用のプロンプトを定義し、共通の `llm` インスタンスと組み合わせて (例: `chain = prompt_template | llm`) LLM呼び出しを行っています。これにより、同じLLMモデルでも異なる指示を与えることで、多様な処理を実現できます。
*   **応用例:**
    *   リサーチアシスタント: 質問受け付け -> 情報検索プロンプトでLLM -> 要約プロンプトでLLM -> 報告書作成プロンプトでLLM。
    *   コード生成・レビュー: 要件定義 -> コード生成LLM -> 生成コード評価LLM -> 修正指示LLM。
    *   このように、タスクを細分化し、各サブタスクに特化したプロンプトを持つLLMノードを連携させることで、より高品質な結果を得ることが期待できます。
---
</details>


### ■ 問題011: 第1章のまとめ - 直列LLMパイプラインの構築

第1章で学んだ様々な要素（状態管理、複数のノード、LLM連携）を組み合わせて、一つの完結した処理パイプラインを構築してみましょう。この問題では、ユーザーからの質問を「分析」「調査」「要約」という3つのステップで処理する、直列（シーケンシャル）なグラフを作成します。

*   **学習内容:** これまで学んだ `StateGraph` の定義、`TypedDict` による状態管理、`add_node` と `add_edge` による直列なグラフ構築、そして各ノードでのLLM呼び出しを統合します。状態が各ノードを通過するたびに段階的に更新され、最終的な成果物が作られていく様子を通じて、LangGraphの基本的なパイプライン構築方法を総復習します。


In [ ]:
# 解答欄011
from typing import TypedDict, Annotated, Optional
from langgraph.graph import StateGraph, ____
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, ____
from langchain_core.prompts import ChatPromptTemplate

# ノートブック冒頭で`llm`変数が初期化されている前提

# --- 状態定義 (State) ---
# パイプライン全体で引き継がれる情報の器を定義
class PipelineState(____):
    messages: Annotated[list, add_messages]
    user_question: str          # ユーザーの元の質問
    analysis_result: str | None # 質問の分析結果
    research_result: str | None # 調査結果
    final_summary: str | None   # 最終的な要約

# --- ノード定義 (Nodes) ---
def capture_question_node(state: PipelineState):
    """最初のノード。ユーザーの質問を状態に保存する。"""
    user_message = state["messages"][-1].content
    return {"user_question": user_message}

def analyze_question_node(state: PipelineState):
    """LLMを使って、質問の主要なトピックや意図を分析するノード。"""
    prompt = ChatPromptTemplate.from_messages([
        ("system", "あなたはユーザーの質問を分析する専門家です。質問の主要なトピックやキーワードを簡潔に抜き出してください。"),
        ("human", "質問: {question}")
    ])
    chain = prompt | llm
    response = chain.invoke({"question": state["user_question"]})
    return {"analysis_result": response.content}

def researcher_node(state: PipelineState):
    """分析結果を基に、LLMが情報を調査・詳述するノード。"""
    prompt = ChatPromptTemplate.from_messages([
        ("system", "あなたは優秀なリサーチャーです。以下のトピックについて、重要なポイントを箇条書きで詳しく説明してください。"),
        ("human", "トピック: {topic}")
    ])
    chain = ____ | llm
    response = chain.invoke({"topic": state["analysis_result"]})
    return {"research_result": response.content}

def summarizer_node(state: PipelineState):
    """調査結果を基に、LLMが最終的な回答を要約するノード。"""
    prompt = ChatPromptTemplate.from_messages([
        ("system", "あなたは情報を分かりやすく要約する専門家です。以下の調査結果を、初心者にも理解できるように丁寧な言葉で要約してください。"),
        ("human", "調査結果:\n{research_info}")
    ])
    chain = prompt | llm
    response = chain.invoke({"research_info": state["research_result"]})
    summary = response.content
    # 最終的な要約を状態に保存し、メッセージ履歴にも追加する
    return {"final_summary": summary, "messages": [AIMessage(content=summary)]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(____)

# ノードをグラフに追加
workflow.add_node("capture_question", capture_question_node)
workflow.add_node("analyzer", analyze_question_node)
workflow.add_node("researcher", researcher_node)
workflow.add_node("summarizer", summarizer_node)

# ノードをエッジで直列につなぐ
workflow.set_entry_point("capture_question")
workflow.add_edge("capture_question", "analyzer")
workflow.add_edge("analyzer", ____)
workflow.add_edge("researcher", "summarizer")
workflow.add_edge(____, END)

# グラフをコンパイル
graph = workflow.compile()

In [ ]:
# --- グラフの可視化 ---
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")


In [ ]:
# --- グラフの実行と結果表示 ---
question = "LangGraphの基本的な仕組みについて、主要な構成要素を挙げて説明してください。"
inputs = {"messages": [HumanMessage(content=question)]}

final_state = graph.invoke(inputs)

print(f"--- 最終的な実行結果 ---")
print(f"■ ユーザーの質問:\n{final_state.get('user_question')}\n")
print(f"■ 質問の分析結果:\n{final_state.get('analysis_result')}\n")
print(f"■ 調査結果:\n{final_state.get('research_result')}\n")
print(f"■ 最終的な要約:\n{final_state.get('final_summary')}")

<details><summary>解答011</summary>

``````python
# 解答011
from typing import TypedDict, Annotated, Optional
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from IPython.display import Image, display

# ノートブック冒頭で`llm`変数が初期化されている前提

# --- 状態定義 (State) ---
# パイプライン全体で引き継がれる情報の器を定義
class PipelineState(TypedDict):
    messages: Annotated[list, add_messages]
    user_question: str          # ユーザーの元の質問
    analysis_result: str | None # 質問の分析結果
    research_result: str | None # 調査結果
    final_summary: str | None   # 最終的な要約

# --- ノード定義 (Nodes) ---
def capture_question_node(state: PipelineState):
    """最初のノード。ユーザーの質問を状態に保存する。"""
    user_message = state["messages"][-1].content
    return {"user_question": user_message}

def analyze_question_node(state: PipelineState):
    """LLMを使って、質問の主要なトピックや意図を分析するノード。"""
    prompt = ChatPromptTemplate.from_messages([
        ("system", "あなたはユーザーの質問を分析する専門家です。質問の主要なトピックやキーワードを簡潔に抜き出してください。"),
        ("human", "質問: {question}")
    ])
    chain = prompt | llm
    response = chain.invoke({"question": state["user_question"]})
    return {"analysis_result": response.content}

def researcher_node(state: PipelineState):
    """分析結果を基に、LLMが情報を調査・詳述するノード。"""
    prompt = ChatPromptTemplate.from_messages([
        ("system", "あなたは優秀なリサーチャーです。以下のトピックについて、重要なポイントを箇条書きで詳しく説明してください。"),
        ("human", "トピック: {topic}")
    ])
    chain = prompt | llm
    response = chain.invoke({"topic": state["analysis_result"]})
    return {"research_result": response.content}

def summarizer_node(state: PipelineState):
    """調査結果を基に、LLMが最終的な回答を要約するノード。"""
    prompt = ChatPromptTemplate.from_messages([
        ("system", "あなたは情報を分かりやすく要約する専門家です。以下の調査結果を、初心者にも理解できるように丁寧な言葉で要約してください。"),
        ("human", "調査結果:\n{research_info}")
    ])
    chain = prompt | llm
    response = chain.invoke({"research_info": state["research_result"]})
    summary = response.content
    # 最終的な要約を状態に保存し、メッセージ履歴にも追加する
    return {"final_summary": summary, "messages": [AIMessage(content=summary)]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(PipelineState)

# ノードをグラフに追加
workflow.add_node("capture_question", capture_question_node)
workflow.add_node("analyzer", analyze_question_node)
workflow.add_node("researcher", researcher_node)
workflow.add_node("summarizer", summarizer_node)

# ノードをエッジで直列につなぐ
workflow.set_entry_point("capture_question")
workflow.add_edge("capture_question", "analyzer")
workflow.add_edge("analyzer", "researcher")
workflow.add_edge("researcher", "summarizer")
workflow.add_edge("summarizer", END)

# グラフをコンパイル
graph = workflow.compile()

# --- グラフの実行と結果表示 ---
question = "LangGraphの基本的な仕組みについて、主要な構成要素を挙げて説明してください。"
inputs = {"messages": [HumanMessage(content=question)]}

final_state = graph.invoke(inputs)

print(f"--- 最終的な実行結果 ---")
print(f"■ ユーザーの質問:\n{final_state.get('user_question')}\n")
print(f"■ 質問の分析結果:\n{final_state.get('analysis_result')}\n")
print(f"■ 調査結果:\n{final_state.get('research_result')}\n")
print(f"■ 最終的な要約:\n{final_state.get('final_summary')}")
``````
</details>


<details><summary>解説011</summary>

#### この問題のポイント

この問題は、第1章で学んだ知識の総仕上げです。条件分岐のような複雑な制御は使わず、LangGraphの最も基本的かつ強力な機能である**「状態（State）を介した直列（シーケンシャル）な処理パイプライン」**を構築します。

*   **段階的な状態の更新:**
    グラフは `capture_question` -> `analyzer` -> `researcher` -> `summarizer` の順に実行されます。`PipelineState`で定義された状態オブジェクト（辞書）が、ノードからノードへとバケツリレーのように渡されていきます。各ノードは、前のノードから渡された状態を読み取り、自身の処理結果を状態に書き加えて次のノードに渡します。
    1.  最初は `user_question` しかありません。
    2.  `analyzer` が `analysis_result` を追加します。
    3.  `researcher` が `research_result` を追加します。
    4.  `summarizer` が `final_summary` と最終的な `messages` を追加して完了です。
    このように、状態が徐々に豊かになっていく様子が、パイプライン処理の本質です。

*   **各ノードの役割分担:**
    このグラフでは、4つのノードがそれぞれ明確な役割を持っています。特に、`analyzer`、`researcher`、`summarizer` の3つのノードは、同じ `llm` インスタンスを使いながらも、`ChatPromptTemplate` を用いてそれぞれ異なる指示（役割）を与えられています。これにより、一つのタスクを複数の専門家（LLM）が連携して解決するような、高度な処理を実現できます。

*   **第1章の総括:**
    この問題を通じて、`StateGraph` と `TypedDict` を使って処理の「状態」を定義し、`add_node` と `add_edge` で処理の「流れ」を組み立てるという、LangGraphの基本的な開発サイクルを体験できました。この直列パイプラインは、LangGraphでアプリケーションを構築する際の最も基本的なパターンであり、今後のより複雑なグラフを設計する上での確かな土台となります。
---
</details>
